In [1]:
# creating a home variable

import os
import glob
import re
import seaborn as sns
import matplotlib.pyplot as plt
import cv2


HOME = os.getcwd()
print(HOME)

/content


In [2]:
# installation of libraries
!pip install roboflow --quiet

from IPython import display
display.clear_output()
from IPython.display import display, Image
from roboflow import Roboflow
import numpy as np

In [3]:
# load the dataset

!mkdir {HOME}/datasets
%cd {HOME}/datasets

rf = Roboflow(api_key="WifAHr5usNgAnytT03tt")
project = rf.workspace("ask-ishan").project("coral-detection-ci")
dataset = project.version(2).download("yolov8")

/content/datasets
loading Roboflow workspace...
loading Roboflow project...
[WARNING] we noticed you are downloading a `yolov8` datasets but you don't have `ultralytics` installed. Roboflow `.deploy` supports only models trained with `ultralytics==8.0.134`, to intall it `pip install ultralytics==8.0.134`.



Extracting Dataset Version Zip to Coral-Detection-CI-2 in yolov8:: 100%|██████████| 271/271 [00:00<00:00, 6063.82it/s]


In [75]:
def get_data_statistics():
  return None

def mask_to_bbox(mask, img_width, img_height):
  x_i = mask[::2]
  y_i = mask[1::2]

  x_min = min(x_i)
  x_max = max(x_i)
  y_min = min(y_i)
  y_max = max(y_i)

  x, x1 = x_min*img_width, x_max*img_width
  y, y1 = y_min*img_height, y_max*img_height
  return [round(x), round(y), round(x1), round(y1)]

In [118]:
data_location = "/content/datasets/Coral-Detection-CI-2/train"

In [119]:
all_label_names = list(glob.iglob(f"{data_location}/labels/*"))

In [120]:
# form main dict with above data location and label names
main_dict = {}

for label_name in all_label_names:
  instance_dict = {}
  with open(label_name) as f:
        textfile = f.read()

  true_classes = [int(i[0]) for i in textfile.strip().split("\n")]
  true_masks = [[float(j) for j in i.split(' ')[1:]] for i in textfile.strip().split("\n")]

  img_easy_name = label_name.split("/")[-1][:-4]

  opened_image = cv2.imread(re.sub("labels", "images", label_name[:-3] + "jpg"))
  opened_image = cv2.cvtColor(opened_image, cv2.COLOR_BGR2RGB)

  height = opened_image.shape[0]
  width = opened_image.shape[1]

  image_size = width*height

  main_dict[img_easy_name] = {"loc":label_name,
                              "instances":{},
                              'image_size':image_size,
                              'image_width':width,
                              'image_height':height}

  for i, j in enumerate(true_masks):
    instance_bbox = mask_to_bbox(j, width, height)
    bbox_size = (instance_bbox[2] - instance_bbox[0]) * (instance_bbox[3] - instance_bbox[1])
    bbox_center = (instance_bbox[0] + (instance_bbox[2]-instance_bbox[0])/2, instance_bbox[1] + (instance_bbox[3]-instance_bbox[1])/2)
    bbox_image = opened_image[instance_bbox[1]:instance_bbox[3], instance_bbox[0]:instance_bbox[2]]
    rgb_colors = np.mean(np.mean(bbox_image[:, :, :]/255, axis=0), axis=0)
    instance_dict[i] = {
        "class": true_classes[i],
        "bbox_size": bbox_size,
        "bbox_size_proportion": bbox_size/image_size,
        "bbox_center": bbox_center,
        "bbox_center_proportion": (bbox_center[1]/(width/2), bbox_center[0]/(height/2)),
        'red_proportion': rgb_colors[0],
        'green_proportion': rgb_colors[1],
        'blue_proportion': rgb_colors[2]
        }
  main_dict[img_easy_name]["instances"] = instance_dict

In [126]:
# gather statistics from main_dict

total_images = len(main_dict.keys())
total_instances = 0
total_class_0 = 0
total_class_1 = 0
bbox_sizes = []
bbox_sizes_proportion = []

for image_key, image_val in main_dict.items():
  total_instances += len(image_val['instances'])
  for instance_val in image_val['instances'].values():
    if instance_val['class'] == 0:
      total_class_0+=1
    else:
      total_class_1+=1
    bbox_sizes.append(instance_val['bbox_size'])
    bbox_sizes_proportion.append(instance_val['bbox_size_proportion'])

In [132]:
print("Total class 1 instances:",total_class_1)
print("Total class 0 instances:", total_class_0)
print('Total number of images', total_images)
print('Total number of instances', total_instances)
print(f"Max, Min, and Avg Bbox sizes:{max(bbox_sizes)}, {min(bbox_sizes)}, {np.mean(bbox_sizes)}")
print(f"Max, Min, and Avg Bbox size Proportions:{max(bbox_sizes_proportion)}, {min(bbox_sizes_proportion)}, {np.mean(bbox_sizes_proportion)}")

Total class 1 instances: 97
Total class 0 instances: 525
Total number of images 98
Total number of instances 622
Max, Min, and Avg Bbox sizes:42622, 410, 5841.948553054663
Max, Min, and Avg Bbox size Proportions:0.13591198979591837, 0.0013073979591836735, 0.01862866247785288
